In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-204/cleaned_df.csv


In [2]:
pip install wurlitzer

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keras-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install tensorflow==2.15.0
!pip install tensorflow-decision-forests==1.8.1
!pip install tensorflow-text==2.15.0
!pip install tf-keras==2.15.1
!pip install torch==2.1.2
!pip install transformers
!pip install wandb

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Hide TensorFlow warnings
os.environ['XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/lib/cuda'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3


In [5]:
import numpy as np
!pip install pandas requests pyarrow tensorflow keras matplotlib scikit-learn nltk

import pandas as pd
import requests
import pyarrow
import matplotlib.pyplot as plt
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import Constant
# from underthesea import word_tokenize
import warnings
warnings.filterwarnings("ignore")
import os
from sklearn.model_selection import train_test_split
# from rouge_score import rouge_scorer
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import string
# from pyvi import ViTokenizer
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
# Đọc tệp CSV vào DataFrame
path = '/kaggle/input/dataset-204/cleaned_df.csv'
df = pd.read_csv(path)

# Hiển thị vài hàng đầu tiên của DataFrame
df.head()

,Unnamed: 0,Summary,Text,cleaned_Text,cleaned_Summary
0,0,Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đ...,"Với bản_tính ham chơi , lười làm , có nhiều ti...",Với bản_tính ham chơi lười làm có nhiều tiền_á...,Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đ...
1,1,Chặn_đứng 2 nhóm giang_hồ mang hung_khí hỗn_ch...,"Mâu_thuẫn từ trước đó , 2 nhóm thanh_niên với ...",Mâu_thuẫn từ trước đó 2 nhóm thanh_niên với số...,Chặn_đứng 2 nhóm giang_hồ mang hung_khí hỗn_ch...
2,2,"Tạm giữ hai đối_tượng tháo ốc , sắt trên đường...","Để có tiền mua ma_tuý , tiêu_xài cá_nhân , Nhậ...",Để có tiền mua ma_tuý tiêu_xài cá_nhân Nhật và...,Tạm giữ hai đối_tượng tháo ốc sắt trên đường_r...
3,3,Tạm giam 4 tháng nữ đồng_nghiệp đâm chết nam t...,"Trong lúc nói_chuyện , Hậu đã tức_giận cầm con...",Trong lúc nói_chuyện Hậu đã tức_giận cầm con d...,Tạm giam 4 tháng nữ đồng_nghiệp đâm chết nam t...
4,4,"Xế hộp tiền tỷ bỗng_dưng "" bốc_hơi "" tại chung...","Xuống nhà_xe chung_cư Hà_Đô lấy xe , một người...",Xuống nhà_xe chung_cư Hà_Đô lấy xe một người_d...,Xế hộp tiền tỷ bỗng_dưng bốc_hơi tại chung_cư ...


In [7]:
df.drop(columns = 'Unnamed: 0')
df.head(5)

,Unnamed: 0,Summary,Text,cleaned_Text,cleaned_Summary
0,0,Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đ...,"Với bản_tính ham chơi , lười làm , có nhiều ti...",Với bản_tính ham chơi lười làm có nhiều tiền_á...,Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đ...
1,1,Chặn_đứng 2 nhóm giang_hồ mang hung_khí hỗn_ch...,"Mâu_thuẫn từ trước đó , 2 nhóm thanh_niên với ...",Mâu_thuẫn từ trước đó 2 nhóm thanh_niên với số...,Chặn_đứng 2 nhóm giang_hồ mang hung_khí hỗn_ch...
2,2,"Tạm giữ hai đối_tượng tháo ốc , sắt trên đường...","Để có tiền mua ma_tuý , tiêu_xài cá_nhân , Nhậ...",Để có tiền mua ma_tuý tiêu_xài cá_nhân Nhật và...,Tạm giữ hai đối_tượng tháo ốc sắt trên đường_r...
3,3,Tạm giam 4 tháng nữ đồng_nghiệp đâm chết nam t...,"Trong lúc nói_chuyện , Hậu đã tức_giận cầm con...",Trong lúc nói_chuyện Hậu đã tức_giận cầm con d...,Tạm giam 4 tháng nữ đồng_nghiệp đâm chết nam t...
4,4,"Xế hộp tiền tỷ bỗng_dưng "" bốc_hơi "" tại chung...","Xuống nhà_xe chung_cư Hà_Đô lấy xe , một người...",Xuống nhà_xe chung_cư Hà_Đô lấy xe một người_d...,Xế hộp tiền tỷ bỗng_dưng bốc_hơi tại chung_cư ...


## mT5

In [8]:
!pip install numpy
!pip install pandas pyarrow requests
!pip install transformers[torch] datasets evaluate accelerate
!pip install dill packaging shapely

import pandas as pd
import pyarrow
import requests
import datasets
import dill
import numpy
import packaging
import shapely

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 942.3 kB/s eta 0:00:00


In [9]:
# Chia dữ liệu theo tỉ lệ 8 : 1 : 1
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df, test_size=0.1,random_state=30)
train_data, val_data = train_test_split(train_data, test_size=0.1,random_state=30)

In [10]:
!pip install transformers[sentencepiece] datasets evaluate
!pip install accelerate

In [11]:
text_word_count = []
summary_word_count = []

for i in df['cleaned_Text']:
      text_word_count.append(len(i.split()))

for i in df['cleaned_Summary']:
      summary_word_count.append(len(i.split()))

max_len_text = round(pd.DataFrame(text_word_count, columns=['word_count']).describe(percentiles=[0.5, 0.7, 0.8, 0.9,0.95]).loc['80%']['word_count'])
max_len_summary = round(pd.DataFrame(summary_word_count, columns=['word_count']).describe(percentiles=[0.5, 0.7, 0.8, 0.9, 0.95]).loc['80%']['word_count'])

In [12]:
pip install -U wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: wandb
    Found existing installation: wandb 0.17.0
    Uninstalling wandb-0.17.0:
      Successfully uninstalled wandb-0.17.0
Note: you may need to restart the kernel to use updated packages.


In [13]:
import wandb
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric
from transformers import DataCollatorForSeq2Seq

# Đăng nhập vào WandB
wandb.login(key="11cd9887d2acce487d73a32f2b8ce3f898c8e5d7")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.7/839.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 91.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.

In [15]:
wandb.init(project="DS204", name="mT5_training")

# Load tokenizer and model
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainer, TrainingArguments, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader

checkpoint = "google/mt5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, legacy=False)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
model.to('cuda')

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["inputs"], max_length=max_len_text, truncation=True, padding=True
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["labels"], max_length=max_len_summary, truncation=True, padding=True
        )
    model_inputs['labels'] = labels['input_ids']
    model_inputs['input_ids'] = model_inputs['input_ids']
    return model_inputs

# Delete later
input_lines = train_data['cleaned_Text'].tolist()
label_lines = train_data['cleaned_Summary'].tolist()

dict_obj = {'inputs': input_lines, 'labels': label_lines}
dataset = Dataset.from_dict(dict_obj)
tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=4)

wandb: Currently logged in as: 21522718 (21522718-University of Information Technology). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.3
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240630_194020-ofw97eub
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run mT5_training
wandb: ⭐️ View project at https://wandb.ai/21522718-University%20of%20Information%20Technology/DS204
wandb: 🚀 View run at https://wandb.ai/21522718-University%20of%20Information%20Technology/DS204/runs/ofw97eub


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/17182 [00:00<?, ? examples/s]

In [16]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, GenerationConfig

# Tạo GenerationConfig với các tham số không mặc định
generation_config = GenerationConfig(
    forced_eos_token_id=2
)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")


training_args = Seq2SeqTrainingArguments("tmp/",do_train=True,do_eval=False,num_train_epochs=3,learning_rate=1e-4,warmup_ratio=0.05,weight_decay=0.01,
        per_device_train_batch_size=2,per_device_eval_batch_size=2,logging_dir='./log',group_by_length=True,save_strategy="epoch",save_total_limit=1, run_name="mT5")

trainer = Seq2SeqTrainer(model=model,args=training_args,train_dataset=tokenized_datasets,data_collator=data_collator,)

trainer.train()
path = "/kaggle/working/tmp/checkpoint-25261"
model.save_pretrained(path)

Step,Training Loss
500,12.967700
1000,3.385000
1500,2.750100
2000,2.506600
2500,2.308800
3000,2.243200
3500,2.140800
4000,2.094200
4500,2.028900
5000,1.959100


In [17]:
pip install rouge_score

  Preparing metadata (setup.py) ... - done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f3d1fbe3ebfaeb96a25f2c95b51c375286774c0055cbd1fff3dde0f471c29e3e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import AutoModelForSeq2SeqLM

# Inference
from datasets import load_metric
metric = load_metric("rouge")

# Delete later
input_lines = test_data['cleaned_Text'].tolist()
label_lines = test_data['cleaned_Summary'].tolist()

dict_obj = {'inputs': input_lines, 'labels': label_lines}

dataset = Dataset.from_dict(dict_obj)
test_tokenized_datasets = dataset.map(preprocess_function, batched=True, remove_columns=['inputs'], num_proc=4)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="pt")

# Path to model checkpoint (adjust based on your setup)
path = "/kaggle/working/tmp/"
checkpoint_file = os.listdir(path)
model = AutoModelForSeq2SeqLM.from_pretrained(f"/kaggle/working/tmp/checkpoint-25261")
model.to('cuda')  # Ensure model is loaded on GPU

Map (num_proc=4):   0%|          | 0/2122 [00:00<?, ? examples/s]

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [19]:
from datasets import load_metric
import torch
import numpy as np
metrics = load_metric('rouge')

max_target_length = max_len_summary
dataloader = torch.utils.data.DataLoader(test_tokenized_datasets, collate_fn=data_collator, batch_size=64)

predictions = []
references = []
for i, batch in enumerate(dataloader):
  outputs = model.generate(
      input_ids=batch['input_ids'].to('cuda'),
      max_length=max_target_length,
      attention_mask=batch['attention_mask'].to('cuda'),
  )
  with tokenizer.as_target_tokenizer():
    outputs = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in outputs]

    labels = np.where(batch['labels'] != -100,  batch['labels'], tokenizer.pad_token_id)
    actuals = [tokenizer.decode(out, clean_up_tokenization_spaces=False, skip_special_tokens=True) for out in labels]
  predictions.extend(outputs)
  references.extend(actuals)
  metrics.add_batch(predictions=outputs, references=actuals)

[{k: v.mid.fmeasure} for k,v in metrics.compute(predictions=predictions, references=references).items()]

[{'rouge1': 0.36818531979756886},
 {'rouge2': 0.18978081537458247},
 {'rougeL': 0.33514802978097125},
 {'rougeLsum': 0.3351367289667359}]

In [20]:
for i,j in zip(references,predictions):
  print('Actual : ', i)
  print('Summary : ', j)
  print(' - ' * 100)

Actual :  Long_An buộc công_ty nước
Summary :  Không có số tiền thu 
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
Actual :  Jun Phạm sử_dụng ch
Summary :  Jun_Phạm tiết_lộ v
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 
Actual :  Bộ VH TT & DL kiến_n
Summary :  Bộ VH TT & DL nói g
 -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  -  - 

In [21]:
import pandas as pd

# Assuming references and predictions are defined
mT5_df = pd.DataFrame({'Actual': references, 'Summary': predictions})

# Replace underscores with spaces in string columns
mT5_df = mT5_df.applymap(lambda x: x.replace('_', ' ') if isinstance(x, str) else x)
mT5_df['Summary'] = mT5_df['Summary'].apply(lambda x: x.strip() + '.' if isinstance(x, str) and not x.endswith('.') else x)

# Save DataFrame to CSV file
output_file = "/kaggle/working/mT5_predictions.csv"
mT5_df.to_csv(output_file, index=False)
mT5_df.head()

,Actual,Summary
0,Long An buộc công ty nước,Không có số tiền thu.
1,Jun Phạm sử dụng ch,Jun Phạm tiết lộ v.
2,Bộ VH TT & DL kiến n,Bộ VH TT & DL nói g.
3,Sợ con trượt tốt,Hà Giang lo con trai tr.
4,Thành lập 6 phường mới thu,Đồng Nai tổ chức.
